In [1]:
import subprocess
import os
import pandas as pd
import openai
import time
import numpy
from utilities.readInfo import read_info
from utilities.embedding import get_embedding
from utilities.create_clone import create_clone, get_clone_path
import chardet

text_file = open("API_key.txt", "r")
openai.api_key =  text_file.read()
text_file.close()

fs = pd.DataFrame()

def get_diff(old_file_path, new_file_path):
    result = subprocess.run(["diff", old_file_path, new_file_path], capture_output=True, text=True)
    return result.stdout

def summarize_str(filename,string):
    while True:
        try:
             response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[{"role": "system", "content": "Summarize what this file in the codebase does, assume context when neccessary."},
                      {"role": "user", "content": "File "+filename+" has "+string}],
                temperature=0,
                max_tokens=256
             )
             return response["choices"][0]["message"]['content']
        except Exception as e:
             print(f"Encountered error: {e}")
             print("Retrying in 20 seconds...")
             time.sleep(20)

def resync(filename):
    root = read_info()
    with open(os.path.join(root, filename), 'rb') as f:
        result = chardet.detect(f.read())
    if not(result['encoding'] == 'ascii' or result['encoding'] == 'ISO-8859-1'):
        print("File "+file+" was not summarised as it is not a text file")
        return "Ignore"
    full_file_path = os.path.join(path, file)
    with open(full_file_path, 'r') as f:
        file_contents = f.read()
    if num_tokens_from_string(file_contents) > 4096:
        print("File too large to summarize. Splitting file into blocks.")
        blocks = split_file_into_blocks(file)
        summaries = []
        for block in blocks:
            summaries.append(summarize_str(file,block))
        return "\n".join(summaries)
    return summarize_str(file,file_contents)

def syncAI():
    path = read_info()
    global fs

    print("Syncing AI :")
    file_paths_details = []
    Files_to_ignore = open(path+"/.AIIgnore", "r").read().splitlines()
    print("Files and directories to ignore:")
    print(Files_to_ignore)

    for root, directories, files in os.walk(path):
        # Exclude any directories that appear in the ignore list
        directories[:] = [d for d in directories if d not in Files_to_ignore]
        print("Directories:", directories)
        for filename in files:
            if filename not in Files_to_ignore:
                #print("Analyzing : "+filename)
                with open(os.path.join(root, filename), 'rb') as f:
                    result = chardet.detect(f.read())
                    #print(result['encoding'])
                if result['encoding'] == 'ascii' or result['encoding'] == 'ISO-8859-1':
                    file_paths_details.append(os.path.join(root, filename))

    # Find the set difference between file_paths_details and df4["filepath"]
    new_file_paths = set(file_paths_details) - set(fs["file_path"])
    # Iterate over the new_file_paths set and create a new row for each file path
    new_rows = []
    for file_path in new_file_paths:
        # Create a dictionary with the values for each column in the new row
        row_dict = {
            "filepath": file_path
            # Add any other columns you need for the new row
        }
        # Append the new row to the new_rows list
        new_rows.append(row_dict)
        print("New File : "+file_path)

    # Convert the new_rows list of dictionaries to a pandas DataFrame
    new_fs = pd.DataFrame(new_rows)
    fs = pd.concat([fs, new_fs], ignore_index=True)
    del_file_paths = set(fs["file_path"]) - set(file_paths_details)

    # Iterate over the del_file_paths set and remove the corresponding rows from dataframes
    for file_path in del_file_paths:
        fs = fs[fs["file_path"] != file_path]
        print("Deleted File : "+file_path)

    for ind in fs.index:
        print(fs['filepath'][ind])
        resync(fs['filepath'][ind])

    if(len(new_fs)>0):
        line_embeddings = []
        for i in range(0,len(new_fs)):
            filename = new_fs.iloc[i][0]
            with open(filename, "r") as f:
                lines = f.readlines()
                line_number = 0
                for line in lines:
                    line_embeddings.append([filename])
                    line_number += 1

        add2fs = pd.DataFrame(line_embeddings)
        add2fs.columns = ["file_path","",""]
        add2df['embedding'] = ''

        i=0
        for ind in add2df.index:
                i+=1
                add2df['embedding'][ind] = get_embedding(add2df['Code'][ind],0.5)
                print(round(100*i/len(add2df)))
        print("Done")

    create_clone(read_info())

    return

syncAI()

Syncing AI :
Files and directories to ignore:
['apiKeys.js']
Directories: ['images']
Directories: []


KeyError: 'file_path'